In [1]:
import os, sys, json
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [2]:
from json import dumps
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

def list_geojson_properties(geojson):
    print('GeoJson properties:\n' + "; ".join(geojson['properties']))
    print('GeoJson geometry coordinates: ' + str(geojson['geometry']['coordinates'][0][0]))

def get_polygon_from_geojson_geometry(geometry):
    polygon_list = list()
    if geometry['type'] == 'MultiPolygon':
        for polygon in geometry['coordinates']:
            polygon_list.append(Polygon(polygon[0]))
    elif geometry['type'] == 'Polygon':
        polygon_list.append(Polygon(geometry['coordinates'][0]))
    return polygon_list

# Read county boundary

In [3]:
BOUNDARIES_NAME = 'domain/la-county-boundary.geojson'
districts_data = dict()
with open(BOUNDARIES_NAME, 'r') as f:
    data = json.load(f)
    district_list = data['features']

for district in district_list:
    district_name = str(district['properties']['OBJECTID'])
    district_name = district_name.replace(' ', '_')
    districts_data[district_name] = dict()
    districts_data[district_name]['coordinates'] = get_polygon_from_geojson_geometry(district['geometry'])

# Read transportation

In [4]:
# import shapefile
# read_transportation = False
# if read_transportation:
#     CITY_FOLDER_NAME = 'la-five'
#     INPUT_FILE_NAME = 'SOCAL10_Emissions_Inventory_500m_Annual_CO2_WGS'
#     OUTPUT_FILE_NAME = 'LA_CO2_WGS'


#     reader = shapefile.Reader(os.path.join(CITY_FOLDER_NAME, INPUT_FILE_NAME + '.shp'))
#     fields = reader.fields[1:]
#     field_names = [field[0] for field in fields]
#     buffer = []
#     count = 0
#     geojson = open(os.path.join(CITY_FOLDER_NAME, OUTPUT_FILE_NAME + '.geojson'), 'w')
#     for sr in reader.shapeRecords():
#         count += 1
#         atr = dict(zip(field_names, sr.record))
#         geom = sr.shape.__geo_interface__
#         buffer.append(dict(type="Feature", geometry=geom, properties=atr))
#         if count % 10000 == 0:
#             sys.stdout.write("\rTotal building processed: %d" % count)
#             sys.stdout.flush()

#     sys.stdout.write("\rTotal building processed: %d" % count)
#     sys.stdout.flush()
#     geojson.write(json.dumps({"type": "FeatureCollection", "features": buffer}, indent=2))
#     geojson.close()


### Mapping transportation grids to WRF grids

In [5]:
with open('transportation-data/transportation-sep/LA_CO2_WGS.geojson', 'r') as f:
    data = json.load(f)
    data_json = data['features']
    trans_grid_data = dict()
    for grid_data in data_json:
        grid_id = str(grid_data['properties']['Id'])
        trans_grid_data[grid_id] = get_polygon_from_geojson_geometry(grid_data['geometry'])

In [6]:
len(trans_grid_data)

171008

In [7]:
trans_grid_data_domain = dict()
count = 0
for grid_id in trans_grid_data:
    count += 1
    poly = trans_grid_data[grid_id]
    for district in districts_data: 
        district_poly = districts_data[district]['coordinates'][0]
        centroid = poly[0].centroid
        if district_poly.contains(centroid) > 0:
            trans_grid_data_domain[grid_id] = centroid
            break
    if count % 10000 == 0:
        sys.stdout.write("\rTotal grid processed: %d" % count)
        sys.stdout.flush()
        
sys.stdout.write("\rTotal grid processed: %d" % count)
sys.stdout.flush()

Total grid processed: 171008

In [8]:
len(trans_grid_data_domain)

41048

In [9]:
print(trans_grid_data_domain['92029'])

POINT (-118.4347229125807 32.81456010610546)


In [11]:
count = 0
found = 0
with open('bldg-wrf-mapping/wrf-grids-origin.geojson', 'r') as f:
    data = json.load(f)
    wrf_grids = data['features']
    for wrf_grid in wrf_grids:
        count += 1
        wrf_poly = get_polygon_from_geojson_geometry(wrf_grid['geometry'])[0]
        for grid_id in trans_grid_data_domain:
            trans_centroid = trans_grid_data_domain[grid_id]
            if wrf_poly.contains(trans_centroid):
                wrf_grid['properties']['trans_grid_id'] = grid_id
                found += 1
                break
        if count % 10 == 0:
            sys.stdout.write("\rTotal grid processed: %d, found: %d" % (count, found))
            sys.stdout.flush()

    if count % 10 == 0:
        sys.stdout.write("\rTotal grid processed: %d, found: %d" % (count, found))
        sys.stdout.flush()

Total grid processed: 130170, found: 29953

In [13]:
out = open('bldg-wrf-mapping/wrf-trans-map.csv', 'w')
ind = -1
row = 384
col = 339
for i in range(row):
    for j in range(col):
        ind += 1
        if 'trans_grid_id' in wrf_grids[ind]['properties']:
            trans_grid_id = wrf_grids[ind]['properties']['trans_grid_id']
        else:
            trans_grid_id = -1
        out.write(str(trans_grid_id) + ",")
    out.write('\n')
out.close()

# Read tranportation emission data and write to files

In [189]:
trans_data = dict()
df = pd.read_csv(os.path.join('transportation-sep', 'Sunday.csv'), 
                 sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

for index, row in df.iterrows():
    trans_data[str(int(row["id"]))] = list(row)[1:]

In [190]:
file_name_prefixs = ["Variable_TRANSHEAT_20090920", "Variable_TRANSHEAT_20090927"]

In [192]:
df_trans = df_trans.interpolate(method='ffill', limit=1)
df_trans = df_trans.fillna(-1)
df_trans.describe()

In [193]:
df_trans.to_csv('bldg-wrf-mapping/wrf-trans-map-grid-fillna.csv')

In [191]:
df_trans = pd.read_csv('bldg-wrf-mapping/wrf-trans-map-grid-fillna.csv', sep=',', parse_dates=True, infer_datetime_format=True, encoding='UTF-8')

In [194]:
for file_name_prefix in file_name_prefixs:
    for hour in range(25):
        file_name = 'transportation-data/wrf-trans-out/' + file_name_prefix + str(hour).zfill(2) + '.txt'
        with open(file_name, 'w') as out:
            for index, row in df_trans.iterrows():
                line = ""
                cur_row = list(row)
                for grid in cur_row:
                    try:
                        line += str(trans_data[str(int(grid))][hour]) + ","
                    except:
                        line += "0.0,"
                out.write(line[:-1] + '\n')